In [12]:
import openai
import json
import os 

from dotenv import load_dotenv
from polywrap_client import PolywrapClient
from typing import cast

from pathlib import Path
from polywrap_client import PolywrapClient, ClientConfig
from polywrap_core import Uri, InvokerOptions, UriPackageOrWrapper, UriResolver
from polywrap_client_config_builder import PolywrapClientConfigBuilder
from polywrap_uri_resolvers import FsUriResolver,SimpleFileReader, StaticResolver, RecursiveResolver
from polywrap_uri_resolvers import UriResolverAggregator
from polywrap_http_plugin import http_plugin


import asyncio

import nest_asyncio

nest_asyncio.apply()

load_dotenv()

ipfs_wrapper_path = Path("fs//Users/robertohenriquez/pycode/polywrap/hackathon/Auto-GPT/autogpt/auto_gpt_workspace/wrappers/ipfs-http-client")


resolver = RecursiveResolver(
        UriResolverAggregator(
            [
                cast(UriResolver, FsUriResolver(file_reader=SimpleFileReader())),
                cast(UriResolver, StaticResolver({Uri("wrap://ens/wraps.eth:http@1.1.0", ipfs_wrapper_path): http_plugin()})),
            ]
        )
    )
config = ClientConfig(resolver=resolver)
client = PolywrapClient(config)

openai.api_key = os.getenv("OPENAI_API_KEY")



def polywrap_bignumber(base_number, factor):
    """Multiplies a base number by a factor to get a new number"""
    uri = Uri.from_str(
        f'fs//Users/robertohenriquez/pycode/cloud/AGI/Auto-GPT/autogpt/auto_gpt_workspace/wrappers/bignumber'
    )
    args = {
        "arg1": base_number,  # The base number
        "obj": {
            "prop1": factor,  # multiply the base number by this factor
        },
    }
    options: InvokerOptions[UriPackageOrWrapper] = InvokerOptions(
        uri=uri, method="method", args=args, encode_result=False
    )
    print(asyncio.run(client.invoke(options)))
    result = asyncio.run(client.invoke(options))
    return f"the result is {result}"


In [14]:

# Step 1, send model the user query and what functions it has access to
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "Find what is 1004 times 919"}],
        functions=[
            {
                "name": "polywrap_bignumber",
                "description": "Multiplies a base number by a factor to get a new number",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "base_number": {
                            "type": "number",
                            "description": "The base number",
                        },
                        "factor": {"type": "number", "description": "The factor to multiply the base number"},
                    },
                    "required": ["base_number", "factor"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]

    # Step 2, check if the model wants to call a function

    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        
        function_args_string = message["function_call"]["arguments"]
        function_args = json.loads(function_args_string)  # Parse the string into a dictionary

        # Step 3, call the function
        # Note: the JSON response from the model may not be valid JSON
        function_response = polywrap_bignumber(
            base_number=function_args.get("base_number"),
            factor=function_args.get("factor"),
        )

        # Step 4, send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What is the weather like in boston?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

print(run_conversation())

WrapAbortError: 
                WrapAbortError: The following wrapper aborted execution with the given message:
                URI: wrap://fs//Users/robertohenriquez/pycode/cloud/AGI/Auto-GPT/autogpt/auto_gpt_workspace/wrappers/bignumber
                Method: method
                Args: {
  "arg1": 1004,
  "obj": {
    "prop1": 919
  }
}
                env: None
                Message: __wrap_abort: Property must be of type 'string'. Found 'uint16'.
  Context: Deserializing module-type: method
    at arg1: BigNumber >> type found, reading property
      at arg1: unknown >> searching for property type
File: ~lib/@polywrap/wasm-as/msgpack/ReadDecoder.ts
Location: [167,5]
